## 這個 colab 示範用純 single agent 做 deep search

In [ ]:
from google.colab import userdata

openai_api_key = userdata.get('openai_api_key')
tavily_api_key = userdata.get('tavily_api_key')
jina_api_key = userdata.get('jina_api_key')

In [ ]:
jina_api_key

'jina_0bf20d126cf84c9aaeda13625970bd3f7CynMUv_yVMwPml57HqKoCa2JpP5'

In [ ]:
!pip install openai-agents

In [ ]:
!pip install tavily-python

In [ ]:
import os
import json
import aiohttp
from pydantic import BaseModel, Field

from agents import Agent, Runner, set_default_openai_key, function_tool, trace
set_default_openai_key(openai_api_key)

from tavily import AsyncTavilyClient

In [ ]:
tavily_client = AsyncTavilyClient(api_key=tavily_api_key)

@function_tool
async def perform_web_search(query: str):
    """Perform web search"""
    print(f"  🔍 進行 web search: {query}")

    search_response = await tavily_client.search(query, max_results=3)
    contexts = search_response["results"]
    # print(f"    Tavily search results: {contexts}")
    return str(contexts)

@function_tool
async def fetch_webpage_text(url: str):
    """Fetch webpage content"""

    print(f"  🌐 抓取 webpage: {url}")
    full_url = f"https://r.jina.ai/{url}"
    headers = {
        "Authorization": f"Bearer {jina_api_key}"
    }
    try:
        async with aiohttp.ClientSession() as session:
            async with session.get(full_url, headers=headers) as resp:
                if resp.status == 200:
                    return await resp.text()
                else:
                    text = await resp.text()
                    print(f"Jina fetch error for {url}: {resp.status} - {text}")
                    return ""
    except Exception as e:
        print(f"從 Jina 獲取網頁內容時發生錯誤：{e}")
        return "fetch error, please ignore this reference"

@function_tool
async def generate_intermediate_answer(answer: str):
    """Generate intermediate answer"""
    print(f"  📝  生成中間答案：{answer}")
    return "ok"

@function_tool
async def generate_sub_questions(sub_questions: str):
    """Generate sub questions """
    print(f"  🧠 生成子問題：{sub_questions}")
    return "ok"

In [ ]:
class SearchResult(BaseModel):
    references: list[str] = Field(
        description="URLs of the most relevant and authoritative sources used to formulate the answer"
    )
    final_answer: str = Field(
        description="Comprehensive response addressing the user's original question based on research findings"
    )

async def run_deep_search(user_query: str, system_prompt: str):
    print(f"💬 用戶問題: {user_query}")

    agent = Agent( name="Deep Search",
                   instructions=system_prompt,
                   model="gpt-4.1-2025-04-14",
                   tools=[generate_sub_questions, perform_web_search, fetch_webpage_text, generate_intermediate_answer],
                   output_type=SearchResult
                 )

    result = await Runner.run(agent, input=user_query, max_turns=30) # 預設只有 10，需要增加!!

    return result.final_output

## 中文初版本

這個版本的流程順序不太正確 :(

In [ ]:
system_prompt_v0 = """您是一個負責深入研究用戶問題的助手，請依照以下步驟進行：

1. 先將用戶問題拆解成多個子問題，使用 generate_sub_questions
2. 針對每個子問題：
  a. 使用 perform_web_search 進行網路搜尋收集相關資料
  b. 使用 fetch_webpage_text 獲取網頁內容以取得詳細資料
  c. 使用 generate_intermediate_answer 為該子問題提供初步回答
3. 完成所有子問題研究後，以繁體中文提供完整答案"""

In [ ]:
user_query = "探討量子計算對區塊鏈安全性的潛在威脅與機遇，特別分析後量子密碼學方案在主流加密貨幣（如比特幣和以太坊）中的實施進展，並評估台灣金融科技公司應如何調整其區塊鏈戰略以應對量子優勢時代的到來。"

result = await run_deep_search(user_query, system_prompt_v0)

💬 用戶問題: 探討量子計算對區塊鏈安全性的潛在威脅與機遇，特別分析後量子密碼學方案在主流加密貨幣（如比特幣和以太坊）中的實施進展，並評估台灣金融科技公司應如何調整其區塊鏈戰略以應對量子優勢時代的到來。
  🧠 生成子問題：1. 量子計算對區塊鏈安全性帶來哪些潛在威脅？
2. 量子計算會帶來哪些機遇促進區塊鏈發展？
3. 主流加密貨幣（如比特幣和以太坊）在後量子密碼學方面的實施現狀如何？
4. 後量子密碼學方案有哪些主流技術，適用於區塊鏈系統？
5. 台灣金融科技公司應如何調整其區塊鏈戰略，以因應未來量子優勢所帶來的挑戰？
  🔍 進行 web search: 量子計算對區塊鏈安全性的威脅
  🔍 進行 web search: 量子計算如何促進區塊鏈機遇
  🔍 進行 web search: 比特幣 以太坊 後量子密碼學 實施現狀
  🔍 進行 web search: 區塊鏈適用的主流後量子密碼學 技術整理
  🔍 進行 web search: 台灣金融科技公司 區塊鏈 量子計算 應對策略
  🌐 抓取 webpage: https://cryptowesearch.com/articles/All-you-need-to-know-about-quantum-computer-and-crypto
  🌐 抓取 webpage: https://www.gate.io/zh-tw/learn/articles/quantum-computing-and-blockchain-types-cost-benefits-and-challenges/6580
  🌐 抓取 webpage: https://www.blocktempo.com/blockchain-security-in-the-quantum-era-down/
  🌐 抓取 webpage: https://mundobytes.com/zh-TW/以太坊-zknox/
  🌐 抓取 webpage: https://ah.lib.nccu.edu.tw/bitstream/140.119/151865/1/T16-006.pdf
  📝  生成中間答案：1. 量子計算對區塊鏈安全性的威脅：
量子電腦主要能以Shor演算法破解橢圓曲線加密（ECC）等主流公私鑰加密，以及以Grover演算法加速破解雜

In [ ]:
result.references

['https://cryptowesearch.com/articles/All-you-need-to-know-about-quantum-computer-and-crypto',
 'https://www.gate.io/zh-tw/learn/articles/quantum-computing-and-blockchain-types-cost-benefits-and-challenges/6580',
 'https://www.blocktempo.com/blockchain-security-in-the-quantum-era-down/',
 'https://ah.lib.nccu.edu.tw/bitstream/140.119/151865/1/T16-006.pdf',
 'https://mundobytes.com/zh-TW/%E4%BB%A5%E5%A4%AA%E5%9D%8A-zknox/']

In [ ]:
print(result.final_answer)

【量子計算對區塊鏈安全性的潛在威脅與機遇】

一、威脅
量子電腦因能以Shor演算法攻破今日主流公私鑰（如橢圓曲線加密ECC）、以Grover演算法大幅加快破解雜湊（如SHA-256），對區塊鏈重大威脅如下：
1. 古老或重複使用的地址（如比特幣P2PK、以太坊早期公開公鑰型）暴露於被量子電腦短時間計算出私鑰的高風險，資產恐瞬間盜竊。
2. 採礦過程的雜湊運算大幅被加速，可能造成51%攻擊、歷史區塊竄改等嚴重網路安全威脅。
3. 經典數位簽章和金鑰管理若未升級至抗量子標準，整體信任基礎將瓦解。

雖然目前量子電腦距實用破解主流鏈安全仍需10~15年（需百萬級qubits），但Google Willow晶片、IBM量子平台等進步已經讓社群提前警覺，必須啟動長期的升級部署。

二、機遇
1. 量子計算可最佳化鏈上共識、挖礦算法、交易驗證流程，提升效率、降低能耗和延展性瓶頸。
2. 採用量子密碼學（如量子密鑰分發QKD）強化資料隱私與身份驗證。
3. 支撐更高階的分散式多方計算、跨鏈操作和複雜智能合約實現。

【後量子密碼學在主流加密貨幣中的實施進展】

1. 比特幣
- 遏止對舊型（公鑰已公開）地址的使用，積極提倡新型P2PKH，鎖定抗量子提案（P2QRH地址、SPHINCS+、FALCON）提升安全。
- 部分計劃討論利用零知識證明（如STARK聚合簽名）提升可擴展性與隱私。

2. 以太坊
- 正透過EIP-7560提案引入Winternitz簽名與零知識證明技術。
- ZKnox團隊已在以太坊虛擬機器（EVM）優化NTT運算、降低後量子簽章驗證的gas費用，並提出EIP-7885希望將後量子驗證變成預編譯加速，有助於規模推廣。
- ERC-4337智能合約錢包、地址隱匿化以延長現有設計在量子時代的韌性。

3. 技術困難
- NIST已標準化Dilithium、FALCON、SPHINCS+三大後量子署名。
- 然而，無論哪種方案，相比當前ECDSA，鏈上公鑰/簽章爆增10~30倍，導致資料庫、gas費用、安全效能三者間難以平衡，尚需測試與優化。

4. 其他區塊鏈
- Hyperledger Fabric現有遷移經驗指出，需自底層（如Golang、憑證鏈管理）配合到應用層（DID整合、混合簽章）展開，應避免單一架構依賴，走向敏捷替換與混合驗證。

【台灣金融科技

順序不對! 我希望是 生成子問題 -> 針對第一題 進行 web search -> 抓取 webpage -> 然後才針對第二題 web search .....

## zero-shot 英文改進版本

In [ ]:
system_prompt_v1 = """You are an agent tasked to thoroughly research user questions by following this specific sequence:

## Required Process Flow

1. First use generate_sub_questions to break down the user's question into sub-questions
2. For EACH sub-question, complete this full cycle before moving to the next sub-question:
   - Use perform_web_search to find information specific to the current sub-question
   - Use fetch_webpage_text (multiple times as needed) to gather detailed information
   - Use generate_intermediate_answer to create a response for that specific sub-question
3. Only after ALL sub-questions have been individually researched and answered, provide a final comprehensive answer to the user's original query in Traditional Chinese (Taiwan)

## Important Guidelines

- Process ONE sub-question completely before moving to the next
- Please keep going until the user's question is completely resolved before ending your turn
- Only terminate your turn when you are sure that ALL sub-questions have been thoroughly researched and answered
- The final answer to the user MUST be in Traditional Chinese (Taiwan)

You MUST plan extensively before each function call, and reflect extensively on the outcomes of the previous function calls.
DO NOT do this entire process by making function calls only, as this can impair your ability to solve the problem and think insightfully.
"""

In [ ]:
user_query = "探討量子計算對區塊鏈安全性的潛在威脅與機遇，特別分析後量子密碼學方案在主流加密貨幣（如比特幣和以太坊）中的實施進展，並評估台灣金融科技公司應如何調整其區塊鏈戰略以應對量子優勢時代的到來。"

result = await run_deep_search(user_query, system_prompt_v1)

💬 用戶問題: 探討量子計算對區塊鏈安全性的潛在威脅與機遇，特別分析後量子密碼學方案在主流加密貨幣（如比特幣和以太坊）中的實施進展，並評估台灣金融科技公司應如何調整其區塊鏈戰略以應對量子優勢時代的到來。
  🧠 生成子問題：1. 量子計算對現有區塊鏈（如比特幣、以太坊）安全性的潛在威脅有哪些？
2. 量子計算可能為區塊鏈技術帶來哪些新機遇？
3. 後量子密碼學方案（Post-Quantum Cryptography, PQC）在主流加密貨幣（如比特幣和以太坊）中的實施進展為何？
4. 台灣金融科技公司針對量子優勢時代，應如何調整其區塊鏈技術與商業戰略？
  🔍 進行 web search: 量子計算對比特幣和以太坊安全性威脅
  🌐 抓取 webpage: https://tenten.co/learning/quantum-computing-bitcoin/
  📝  生成中間答案：量子計算對現有區塊鏈（如比特幣、以太坊）的核心安全威脅，主要來自於可用於破解現今主流加密演算法的潛力。例如，比特幣與以太坊廣泛使用的橢圓曲線數位簽名演算法（ECDSA 256）和SHA-256雜湊演算法，都可能被量子電腦藉由Shor演算法與Grover演算法大幅提升破解效率。若未升級抗量子加密，早期使用P2PK格式的比特幣最為脆弱，約有25%的比特幣流通量暴露於此威脅中。雖然現有量子硬體仍需數百萬量子位元才能對這些加密機制造成致命威脅，專家一致認為5~10年內會出現初步風險。因此，必須及早推動後量子密碼學與量子抗性技術的更新與轉型。
  🔍 進行 web search: 量子計算為區塊鏈帶來的新機遇
  🌐 抓取 webpage: https://vocus.cc/article/676a2bacfd89780001961674
  📝  生成中間答案：量子計算雖然對區塊鏈安全性構成潛在威脅，但同時也帶來許多新機遇。首先，量子計算可以大幅提升區塊鏈交易驗證及共識機制的效率，使分散式帳本的資料處理及交易速度顯著增加。其次，量子技術有望催生「量子區塊鏈」，結合量子加密、量子通訊等特性，構築更安全、真正意味上的去中心化帳本基礎架構。此外，量子計算和區塊鏈跨域整合為金融科技帶來革新契機，例如加速複雜金融產品自動化清算、優化大數據隱私協議等，有助於驅動整體產業升級。
  🔍 進行 web se

In [ ]:
result.references

['https://cryptowesearch.com/articles/All-you-need-to-know-about-quantum-computer-and-crypto',
 'https://kknews.cc/zh-tw/tech/4z4ep5g.html',
 'https://rcs.com.tw/%E9%87%8F%E5%AD%90%E5%8D%80%E5%A1%8A%E9%8F%88%EF%BC%9A%E7%B5%90%E5%90%88%E9%87%8F%E5%AD%90%E5%AF%86%E7%A2%BC%E5%AD%B8%E8%88%87%E5%8D%80%E5%A1%8A%E9%8F%88%E6%8A%80%E8%A1%93%E7%9A%84%E9%9D%A9%E5%91%BD/',
 'https://www.blocktempo.com/blockchain-security-in-the-quantum-era-down/',
 'https://www.cio.com.tw/quantum-finance-hits-opportunities-challenges-and-responses-in-the-financial-sector/']

In [ ]:
print(result.final_answer)

【量子計算對區塊鏈安全性的潛在威脅與機遇—主流加密貨幣的後量子實施現況，及台灣金融科技回應建議】

一、量子計算的潛在威脅
量子電腦發展將可突破目前區塊鏈所依賴的橢圓曲線簽章（ECC）和雜湊演算法（SHA-256）等安全基礎。Shor與Grover演算法能大幅加速破解私鑰與雜湊，讓駭客有可能盜取錢包資產、發動51%攻擊、竄改區塊鏈資料，尤其對早期未隱藏公鑰的舊址風險更高。預計最快2030-2035年間將具「實質威脅」，現今量子電腦尚不具攻擊能力，但主流社群已開始提前布局。

二、量子帶來的新機遇與創新模式
量子時代不僅是挑戰，也是創新基礎設施的機會。量子密鑰分配（QKD）可實現理論上無條件的密鑰安全；量子糾纏等物理現象有望用於鏈上更高效率的即時同步與共識機制；量子隨機數生成、不可克隆特性可重塑NFT及隨機應用的新標準。量子區塊鏈已成為國際熱點，多國積極投入實驗研究。

三、後量子密碼學在主流加密貨幣實施進展
美國NIST已完成一輪後量子加密演算法（如Dilithium、Falcon、SPHINCS+）的標準化，Algorand已率先導入Falcon簽名於跨鏈應用，以太坊則有Falcon實驗與聚合簽章技術研究。比特幣、以太坊主網尚未大規模實施，主要因新算法簽章、金鑰體積巨大，將使鏈體膨脹、gas費用劇增、影響去中心化。目前對策多在測試、壓縮與聚合技術的研究，需待標準與效能最適化及社群共識形成。

四、台灣金融科技產業應對量子優勢建議
1. 主動追蹤量子計算及密碼學國際標準進展，及早設計區塊鏈升級藍圖。
2. 優先於新應用先導入後量子密碼體系、混合簽章方案，保留漏洞回滾與升級彈性。
3. 避免重複使用舊型地址與暴露公鑰，推行動態短時簽名與聚合簽章錢包升級。
4. 建立多雲、分層資安、分散式控管架構，降低全域量子攻擊單點風險。
5. 積極參與量子資安產學與產業聯盟，加強跨部門資訊流通與危機應變。
6. 對於金融核心資產，設置過渡期間多重防線（如關鍵交易線上雙密碼、快速回滾等）。

結論是：量子計算短期雖非取代性威脅，但科技與政策需以「預防性安全規範」為前導，臺灣金融科技公司宜即刻啟動量子資安評估、升級規劃與測試鏈推行，確保進入量子時代時能維持信任、創新與競爭力。

參考來源：
- https://cryptowesearch.com/articles/All-you-ne

## few-shot 版本

可以增強 模型遵循流程指示

In [ ]:
system_prompt_v2 = """You are an agent tasked to thoroughly research user questions by following this specific sequence:

## Required Process Flow

1. First use generate_sub_questions to break down the user's question into sub-questions
2. For EACH sub-question, complete this full cycle before moving to the next sub-question:
   - Use perform_web_search to find information specific to the current sub-question
   - Use fetch_webpage_text (multiple times as needed) to gather detailed information
   - Use generate_intermediate_answer to create a response for that specific sub-question
3. Only after ALL sub-questions have been individually researched and answered, provide a final comprehensive answer to the user's original query in Traditional Chinese (Taiwan)

## Tool Sequence Example

- generate_sub_questions
- perform_web_search (for sub-question 1)
- fetch_webpage_text (relevant page 1)
- fetch_webpage_text (relevant page 2)
- fetch_webpage_text (relevant page 3)
- generate_intermediate_answer (for sub-question 1)
- perform_web_search (for sub-question 2)
- fetch_webpage_text (relevant page 1)
- fetch_webpage_text (relevant page 2)
- fetch_webpage_text (relevant page 3)
- generate_intermediate_answer (for sub-question 2)
- [Continue this pattern for all sub-questions]

## Important Guidelines

- Process ONE sub-question completely before moving to the next
- Please keep going until the user's question is completely resolved before ending your turn
- Only terminate your turn when you are sure that ALL sub-questions have been thoroughly researched and answered
- The final answer to the user MUST be in Traditional Chinese (Taiwan)

You MUST plan extensively before each function call, and reflect extensively on the outcomes of the previous function calls.
DO NOT do this entire process by making function calls only, as this can impair your ability to solve the problem and think insightfully.
"""

In [ ]:
user_query = "探討量子計算對區塊鏈安全性的潛在威脅與機遇，特別分析後量子密碼學方案在主流加密貨幣（如比特幣和以太坊）中的實施進展，並評估台灣金融科技公司應如何調整其區塊鏈戰略以應對量子優勢時代的到來。"

result = await run_deep_search(user_query, system_prompt_v2)

💬 用戶問題: 探討量子計算對區塊鏈安全性的潛在威脅與機遇，特別分析後量子密碼學方案在主流加密貨幣（如比特幣和以太坊）中的實施進展，並評估台灣金融科技公司應如何調整其區塊鏈戰略以應對量子優勢時代的到來。
  🧠 生成子問題：1. 量子計算對區塊鏈安全性產生的潛在威脅有哪些？2. 量子計算將帶來哪些機遇促進區塊鏈發展？3. 後量子密碼學有哪些主要方案，以及在比特幣和以太坊中的實施進展如何？4. 台灣金融科技公司應如何調整其區塊鏈戰略來因應量子優勢時代？
  🔍 進行 web search: 量子計算對區塊鏈安全性產生的潛在威脅
  🌐 抓取 webpage: https://kknews.cc/zh-hk/tech/zejm54q.html
  🌐 抓取 webpage: https://cryptowesearch.com/articles/All-you-need-to-know-about-quantum-computer-and-crypto
  🌐 抓取 webpage: https://www.blocktrend.today/p/ep280
  📝  生成中間答案：### 子問題1：量子計算對區塊鏈安全性產生的潛在威脅有哪些？

1. **私鑰破解威脅**：傳統區塊鏈（如比特幣、以太坊）廣泛使用橢圓曲線密碼學（如ECDSA）和SHA-256哈希函數。量子電腦透過Shor演算法可在短時間內解開ECDSA生成的私鑰，讓攻擊者能大規模竊取資產。
2. **挖礦和網絡安全威脅**：Grover演算法能夠顯著加速對SHA-256等雜湊演算法的搜尋。雖然並非直接破解，但可能使得量子電腦主導大部分算力，威脅到區塊鏈的PoW（Proof-of-Work）協議穩定性。
3. **歷史資料暴露風險**：比特幣上早期多次重複使用的P2PK或P2PKH等地址，其公鑰資訊已公開於區塊鏈。一旦量子計算足夠強大，上述老地址尤其容易被破解，資產被盜風險高。
4. **密碼學技術全面過時**：不只是加密貨幣，現今大量互聯網加密技術（如TLS/SSL）都將面臨類似風險，密碼學基礎運算需全面升級。

整體來看，量子計算對於現存區塊鏈最核心的安全假設造成系統性威脅。
  🔍 進行 web search: 量子計算技術對區塊鏈帶來的正面機遇與潛力
  🌐 抓取 webpage: https://

In [ ]:
result.references

['https://kknews.cc/zh-hk/tech/zejm54q.html',
 'https://cryptowesearch.com/articles/All-you-need-to-know-about-quantum-computer-and-crypto',
 'https://www.blocktrend.today/p/ep280',
 'https://www.gate.io/zh-tw/learn/articles/quantum-computing-and-blockchain-types-cost-benefits-and-challenges/6580',
 'https://mundobytes.com/zh-TW/%E4%BB%A5%E5%A4%AA%E5%9D%8A-zknox/',
 'https://www.technice.com.tw/issues/blockchain/156089/']

In [ ]:
print(result.final_answer)

綜合分析量子計算對區塊鏈安全性、後量子密碼學發展現況與台灣金融科技因應之道如下：

一、量子計算對區塊鏈的威脅
量子計算因Shor演算法等能力，未來可於極短時間內破解現行基於RSA、ECDSA、SHA-256等加密技術，對比特幣與以太坊等主流區塊鏈構成三大安全挑戰：（1）公私鑰私有金鑰可被逆推出，造成資產盜竊風險；（2）加密貨幣挖礦及整體網路可能被量子電腦主導和操控；（3）大量早期重複使用的老地址面臨歷史數據暴露危機。

二、量子計算帶來的區塊鏈機遇
除了安全威脅，量子計算也帶來可觀機遇：（1）推動量子密碼學（如QKD）、後量子密碼學在區塊鏈落地，可提升資料隱私與不可變安全性；（2）協助優化交易處理、共識機制及能源消耗，提升可擴展性與效率；（3）量子多方計算、量子隨機數等新技術，有助於Dapps與新一代智能合約發展。

三、後量子密碼學主流方案與落地進展
主力方向包括基於格（如Kyber、Dilithium）、哈希（如XMSS、SPHINCS+）、碼（McEliece）、多變量等。比特幣目前多停留在標準研究與演算法討論，技術推廣如P2QRH這類抗量子地址、引入SPHINCS+、FALCON均在設計驗證階段。以太坊則由ZKnox等團隊推進NTT硬體優化與EIP-7885預編譯提案，目標降低gas費、便於後量子簽章大規模應用。全球皆預計隨NIST標準化步伐於2030年前完成主體升級。

四、台灣金融科技公司戰略建議
台灣金融科技公司應：
1. 盤點現有區塊鏈業務加密基礎，及早評估風險；
2. 優先引入後量子密碼技術（混合加密、SPHINCS+、Kyber等）於重要資產與合約；
3. 定期安全測試並追蹤國際標準、一流公鏈升級趨勢，積極參與產業聯盟或標準組織；
4. 強化工程團隊量子資安訓練，建立內部快速反應機制；
5. 落實多層次防護（如雙重認證、自動換鑰、滲透測試等），保障過渡期穩定。

總結來說，量子時代來襲雖對區塊鏈產業演進與金融科技公司治理帶來巨大挑戰，但也是台灣業者技術升級、規則創新及國際連結的關鍵契機，需積極備戰方能持續立於不敗。

參考資料：
1. 利用量子計算解決區塊鏈的安全威脅｜每日頭條
2. CryptoWesearch｜量子電腦對比特幣真實威脅大解析
3. 區塊勢Podcast｜量子電腦破解比特幣？
4. Gate.io｜量子計算與區塊鏈：類型、成本

## Pseudocode 版本

另一種增強 模型遵循流程指示 的思路

In [ ]:
system_prompt_v3 = """You are an agent tasked to thoroughly research user questions by following this specific sequence:

## Required Process Flow

1. First use generate_sub_questions to break down the user's question into sub-questions
2. For EACH sub-question, complete this full cycle before moving to the next sub-question:
   - Use perform_web_search to find information specific to the current sub-question
   - Use fetch_webpage_text (multiple times as needed) to gather detailed information
   - Use generate_intermediate_answer to create a response for that specific sub-question
3. Only after ALL sub-questions have been individually researched and answered, provide a final comprehensive answer to the user's original query in Traditional Chinese (Taiwan)

## Example (Pseudocode)

Below is an example of the tool usage flow represented as pseudocode:

```
function process_user_question(user_query):
    // Step 1: Break down the user question into sub-questions
    sub_questions = generate_sub_questions(user_query)
    all_intermediate_answers = []

    // Step 2: Process each sub-question one by one
    for each sub_question in sub_questions:
        // Search for information about the current sub-question
        search_results = perform_web_search(sub_question)
        detailed_information = []

        // Get detailed content for each search result
        for each result in search_results:
            if is_relevant(result):
                webpage_content = fetch_webpage_text(result.url)
                detailed_information.append(webpage_content)

        // Generate intermediate answer for the current sub-question
        intermediate_answer = generate_intermediate_answer(sub_question, detailed_information)
        all_intermediate_answers.append(intermediate_answer)

    // Step 3: Integrate all intermediate answers into a final answer
    final_answer = synthesize_comprehensive_answer(user_query, all_intermediate_answers)
    return translate_to_traditional_chinese(final_answer)
```

## Important Guidelines

- Process ONE sub-question completely before moving to the next
- Please keep going until the user's question is completely resolved before ending your turn
- Only terminate your turn when you are sure that ALL sub-questions have been thoroughly researched and answered
- The final answer to the user MUST be in Traditional Chinese (Taiwan)

You MUST plan extensively before each function call, and reflect extensively on the outcomes of the previous function calls.
DO NOT do this entire process by making function calls only, as this can impair your ability to solve the problem and think insightfully.
"""

In [ ]:
user_query = "探討量子計算對區塊鏈安全性的潛在威脅與機遇，特別分析後量子密碼學方案在主流加密貨幣（如比特幣和以太坊）中的實施進展，並評估台灣金融科技公司應如何調整其區塊鏈戰略以應對量子優勢時代的到來。"

result = await run_deep_search(user_query, system_prompt_v3)

💬 用戶問題: 探討量子計算對區塊鏈安全性的潛在威脅與機遇，特別分析後量子密碼學方案在主流加密貨幣（如比特幣和以太坊）中的實施進展，並評估台灣金融科技公司應如何調整其區塊鏈戰略以應對量子優勢時代的到來。
  🧠 生成子問題：1. 量子計算對區塊鏈安全性帶來哪些主要潛在威脅？
2. 量子計算對區塊鏈技術帶來哪些機遇？
3. 後量子密碼學方案在比特幣中的實施進展如何？
4. 後量子密碼學方案在以太坊中的實施進展如何？
5. 台灣金融科技公司面對量子優勢時代，應如何調整其區塊鏈戰略？
  🔍 進行 web search: 量子計算對區塊鏈安全性的主要潛在威脅
  🌐 抓取 webpage: https://www.cw.com.tw/article/5133212
  📝  生成中間答案：量子計算對區塊鏈安全性的主要潛在威脅在於其對目前主流加密演算法（如比特幣所使用的ECDSA 256與SHA-256）可能造成的破解風險。現今量子電腦還未能真正威脅到這些加密技術，但若未來能運算超過百萬甚至數百萬量子位的量子電腦問世，將有理論上的能力破解現有加密私鑰與雜湊功能，對舊式比特幣（P2PK），以及交易過程中暴露公鑰的用戶資產安全產生嚴重威脅。不過，目前這樣的量子能力還需很長的時間、技術突破與巨額投資才能實現。比特幣創建初期即有預警這類風險，並提出異動演算法與分歧（fork）的應對方案，以維持區塊鏈長期安全性。結論是，量子計算不會在短期、甚至中期內對區塊鏈安全造成致命性衝擊，但這確實是一項中長期需正視的威脅。參考：天下雜誌、專家訪談、Google最新研究。網址：https://www.cw.com.tw/article/5133212
  🔍 進行 web search: 量子計算對區塊鏈技術帶來的機遇
  🌐 抓取 webpage: https://www.gate.io/zh-tw/learn/articles/quantum-computing-and-blockchain-types-cost-benefits-and-challenges/6580
  📝  生成中間答案：量子計算對區塊鏈技術帶來的主要機遇包括：

1. 增強區塊鏈效率與可擴展性：量子計算可優化共識機制、交易驗證與資源分配，有潛力顯著提升區塊鏈的運算效率與網路擴展能力，並減少挖礦及交易過程中的能源消耗。


In [ ]:
result.references

['https://www.cw.com.tw/article/5133212',
 'https://www.gate.io/zh-tw/learn/articles/quantum-computing-and-blockchain-types-cost-benefits-and-challenges/6580',
 'https://www.blocktempo.com/adam-back-talks-about-quantum-computing/',
 'https://notes.kurtpan.com/ethdilithiumfalcon',
 'https://www.cio.com.tw/quantum-finance-hits-opportunities-challenges-and-responses-in-the-financial-sector/']

In [ ]:
print(result.final_answer)

【量子計算對區塊鏈安全性的威脅與機遇】

1. 量子計算的安全威脅：
量子計算具備以前所未有的平行運算能力，未來若達到百萬~數百萬量子位元以上，有潛力破解目前主流區塊鏈（如比特幣、以太坊）所依賴的關鍵加密演算法（ECDSA、SHA-256）。這威脅尤以早期比特幣（如P2PK模式）最為明顯，未來若無升級將衍生大規模公私鑰失竊、資產不保等嚴重後果。不過，目前量子電腦尚未足以構成直接衝擊，專家預估要數十年才會達到真正破解門檻。

2. 量子計算的產業機遇：
量子運算亦為區塊鏈產業帶來新契機，包括：
- 共識機制、交易驗證、智能合約的運算與效率最佳化，解決目前公鏈瓶頸如處理速度與能效問題。
- 以量子密鑰分發（QKD）、安全多方運算等前沿方案提升資訊隱私與信任層級。
- 區塊鏈與量子計算的協同創新，產生新型量子增強型區塊鏈架構。

【比特幣、以太坊後量子密碼學進展】

▪ 比特幣：
比特幣社群對抗量子威脅已有高度意識。現行協議（如Taproot升級）已預留未來可軟分叉導入後量子簽名的技術空間，未來若量子威脅迫切，理論上社群共識可推動主網以軟分叉快速引進抗量子密碼學。例如未來可導入Schnorr或更高防護等級的後量子簽章，並保留現有鏈效率與兼容性。
（參考：https://www.blocktempo.com/adam-back-talks-about-quantum-computing/）

▪ 以太坊：
以太坊社群與多家技術團隊已積極開發格基底的簽名方案（如FALCON、DILITHIUM）。ZKNOX 等團隊已完成多種方案的鏈上工程原型，並依以太坊合約運算限制極力優化gas成本。短期尚未大規模佈署到主鏈，但部分Layer2、隱私支付實驗系統已採先進後量子簽名試水溫。發生量子威脅時，相關技術可望快速納入主網標準。
（參考：https://notes.kurtpan.com/ethdilithiumfalcon）

【台灣金融科技公司區塊鏈戰略建議】

面對量子計算帶來的變局，台灣金融科技公司建議採取如下戰略：
1. 嚴格評估現有金融區塊鏈平台與加密協議的量子風險，提前規劃升級藍圖。
2. 追蹤並對標主流公鏈後量子標準（如NIST、BTC/ETH發展路徑），參加測試與技術驗證。
3. 編組跨部門資安／密碼學專案小組，與監理單位及學界合作建立量子資安演練情境。
4. 強化資料